# Imports

In [1]:
!pip install tensorflow

In [2]:
import os
import numpy as np
import tensorflow as tf

from keras.callbacks import TensorBoard
from time import strftime
from PIL import Image

%matplotlib inline

In [3]:
from numpy.random import seed
seed(777)

tf.random.set_seed(888)

# **Constants**

In [4]:
LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# **Get the Data**

In [5]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [8]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Explore


In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [14]:
# Re-scale our features
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### Convert target values to one-hot encoding

In [15]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
values

array([5, 0, 4, 1, 9])

In [18]:
values[4]

9

In [19]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [20]:
y_train_all.shape

(60000, 10)

In [21]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [22]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [23]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [24]:
x_train.shape

(50000, 784)

In [25]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [26]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [27]:
nr_epochs = 50
# can adjust learning rate to see changes!
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [28]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.compat.v1.random.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
        
        return layer_out

In [29]:
# Model without dropout

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [30]:
# Model with dropout

layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [31]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H.%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directory')

Successfully created directory


# Loss, Optimization & Metrics

#### Defining Loss Function

In [32]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

#### Defining Optimizer

In [33]:
with tf.name_scope('optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

In [34]:
#tf.compat.v1.reset_default_graph()

#### Accuracy Metric

In [35]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.math.equal(tf.math.argmax(output, axis=1), tf.math.argmax(Y, axis=1))
    accuracy = tf.math.reduce_mean(tf.cast(correct_pred, tf.float32))

In [36]:
with tf.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [37]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [38]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [39]:
merged_summary = tf.compat.v1.summary.merge_all()

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

#### Initialize all the variables

In [40]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

# Batching the Data

In [41]:
size_of_batch = 1000

In [42]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [43]:
nr_iterations

50

In [44]:
from tensorflow.python.ops.gen_linalg_ops import batch_self_adjoint_eig
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += index_in_epoch + batch_size 
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Loop

In [45]:
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =============
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ==================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.817062497138977
Epoch 1 	| Training Accuracy = 0.8423749804496765
Epoch 2 	| Training Accuracy = 0.8550000190734863
Epoch 3 	| Training Accuracy = 0.862500011920929
Epoch 4 	| Training Accuracy = 0.9521874785423279
Epoch 5 	| Training Accuracy = 0.9618124961853027
Epoch 6 	| Training Accuracy = 0.9670000076293945
Epoch 7 	| Training Accuracy = 0.9709374904632568
Epoch 8 	| Training Accuracy = 0.9751874804496765
Epoch 9 	| Training Accuracy = 0.9784374833106995
Epoch 10 	| Training Accuracy = 0.9806249737739563
Epoch 11 	| Training Accuracy = 0.9819375276565552
Epoch 12 	| Training Accuracy = 0.9831249713897705
Epoch 13 	| Training Accuracy = 0.9862499833106995
Epoch 14 	| Training Accuracy = 0.9866874814033508
Epoch 15 	| Training Accuracy = 0.9880625009536743
Epoch 16 	| Training Accuracy = 0.9885625243186951
Epoch 17 	| Training Accuracy = 0.9896249771118164
Epoch 18 	| Training Accuracy = 0.9901250004768372
Epoch 19 	| Training Accuracy = 0.990875005

# Make a Prediction

In [46]:
img = Image.open('MNIST/test_img.png')
img

In [47]:
bw = img.convert('L')

In [48]:
img_array = np.invert(bw)

In [49]:
img_array.shape

(28, 28)

In [50]:
test_img = img_array.ravel()

In [51]:
test_img.shape

(784,)

In [52]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [53]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

In [54]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.50%


# Reset for the Next Run

In [55]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()

# CMD Virtual Environment for Tensorboard

In [56]:
# python -m venv --system-site-packages .\venv

# .\venv\Scripts\activate

# tensorboard --logdir='filepath'

# deactivate  # don't exit until you're done using TensorFlow

# Code for 'Hyperparameter' section before refactoring

In [57]:
# with tf.name_scope('hidden_layer_1'):

#     initial_w1 = tf.compat.v1.random.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [58]:
# with tf.name_scope('hidden_layer_2'):

#     initial_w2 = tf.compat.v1.random.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [59]:
# with tf.name_scope('output_layer'):
    
#     initial_w3 = tf.compat.v1.random.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')
    
#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')
    
#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [60]:
# b3.eval(sess)